In [ ]:
# !pip install tensorflow
# !pip install pandas
# !pip install matplotlib
# !python -m pip install numpy scipy matplotlib ipython jupyter pandas sympy nose

In [ ]:
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

import os, datetime, random
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
# memory 관련 이슈를 해결하기 위한 코드
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
# set random seed
# np.random.seed(42)
random.seed(12345)
tf.random.set_seed(42)

In [ ]:
# 사전 설정을 위한 세팅
epochs = 256
image_size = 256
batch = 32

# Directory check
logdir = '../logs'
if not os.path.isdir(logdir):
    os.mkdir(logdir)
    os.mkdir(logdir + "/disease")
    os.mkdir(logdir + "/plant")

In [ ]:
# Read dataframe from tsv file
train_df = pd.read_csv("../data/train/train.tsv", sep='\t', header=None)
train_df.columns = ['filename', 'plant', 'disease']

test_df = pd.read_csv("../data/test/test.tsv", sep='\t', header=None)
test_df.columns = ['filename']

# for categorical labels
train_df[['plant', 'disease']] = train_df[['plant', 'disease']].astype(str)
train_df["label"] = train_df['plant'] + "_"+train_df['disease']

In [ ]:
# declare image generator
train_image_generator = ImageDataGenerator(rotation_range=5, horizontal_flip=True, validation_split=0.25) # Generator for our training, validation data
test_image_generator = ImageDataGenerator() # Generator for our test data

In [ ]:
train_data_gen = train_image_generator.flow_from_dataframe(dataframe=train_df,
                                                           directory="../data/train",
                                                           x_col="filename",
                                                           y_col="label",
                                                           class_mode="categorical",
                                                           subset='training',
                                                           shuffle=True,
                                                           target_size=(image_size,image_size),
                                                           batch_size=batch)

valid_data_gen = train_image_generator.flow_from_dataframe(dataframe=train_df,
                                                           directory="../data/train",
                                                           x_col="filename",
                                                           y_col="label",
                                                           class_mode="categorical",
                                                           subset='validation',
                                                           shuffle=True,
                                                           target_size=(image_size,image_size),
                                                           batch_size=batch)

test_data_gen = test_image_generator.flow_from_dataframe(dataframe=test_df,
                                                         directory="../data/test",
                                                         x_col="filename",
                                                         y_col=None,
                                                         class_mode=None,
                                                         shuffle=False,
                                                         target_size=(image_size,image_size),
                                                         batch_size=batch)

In [ ]:
sample_training_images, _ = next(train_data_gen)

# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img/225)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
    
plotImages(sample_training_images[:5])

In [ ]:
# Call bottom model
inputs = Input(shape = (image_size, image_size, 3))
bot_model = EfficientNetB2(weights='imagenet', input_tensor=inputs, classifier_activation=None, include_top=False)
# Rebuild top
x = GlobalAveragePooling2D(name="avg_pool")(bot_model.output)
x = BatchNormalization(name="bn_avg_pool")(x)
x = Dropout(0.5)(x)
x = Dense(176, activation="swish", name="top_dense_1")(x)
x = Dense(44, activation="swish", name="top_dense_2")(x)
outputs = Dense(20, activation="softmax", name="pred")(x)

In [ ]:
# Compile
model = tf.keras.Model(inputs, outputs, name="EfficientNet")
model.compile(optimizer=Adam(learning_rate=0.000002),
loss='categorical_crossentropy',
metrics=['accuracy'])
logdir = os.path.join("../logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
model.summary()

In [ ]:
callbacks = [
    ModelCheckpoint(
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        # The saved model name will include the current epoch.
        filepath="../model/mymodel_best.h5",
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="val_loss",
        verbose=1,
    ),
    EarlyStopping(
        monitor="val_accuracy",
        min_delta=1e-4,
        patience=16,
        verbose=1,
    ),
    TensorBoard(logdir, histogram_freq=1)
]

In [ ]:
history = model.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=valid_data_gen,
    callbacks=callbacks,
)

In [ ]:
label_pred = model.predict(test_data_gen)
test_df['label'] = tf.math.argmax(label_pred, 1)

In [ ]:
class_indices = train_data_gen.class_indices
class_indices = dict([(value, key) for key, value in class_indices.items()])

In [ ]:
test_df = test_df.replace({"label": class_indices})

In [ ]:
test_df["plant"] = test_df.label.str.split("_").str[0]
test_df["disease"] = test_df.label.str.split("_").str[1]
test_df = test_df.drop(columns = "label")

In [ ]:
test_df.to_csv("new_test_df.tsv", sep='\t', header = False, index=False)